# nb-model

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_xg=test-prep-transpose.ipynb'   # FILL
dir_name = 'model'             # FILL
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
import torch

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, is_type, pd_common_idx_rows, remove_dups_list, set_loglevel, search_df, chained_filter, get_variants, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import midx_get_level, pd_rows, midx_intersect, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_is_ndim, identity_fn
from model.common import DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, EXPECTED_NUM_HOURS, default_dataset
from model.dataprep_util import align_first_last_cols, prune_nulls
from model.datagen_util import xgdg, process_group
from data.data_api import DataAPI
from recon.dataset_util import prep_dataset, gen_group
from model.dataprep_util import COMMON_PREP_MAPPING, DATA_PREP_MAPPING, prep_transpose_data, align_first_last_cols
#from recon.split_util import get_train_test_split, gen_time_series_split, index_three_split, pd_binary_clip
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_xg=test-prep-transpose.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


## Load

In [2]:
dohlca_rcs, dohlca_dfs = DataAPI.axe_load(['hohlca', 'hohlca_pba_vol'])
hrm_rcs, hrm_dfs = DataAPI.axe_load(['hrm', 'hrm'])

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [59]:
asset_name = 'sp_500'

In [60]:
d = dohlca_dfs[[asset_name, 'hohlca', 'hohlca_pba_vol', 'pba_hohlca', 'pba_hohlca']]
r = hrm_dfs[[asset_name, 'hrm', 'hrm', 'pba', 'pba']]

## Test FN

In [61]:
from common_util import is_type, compose, dcompose, pd_idx_rename, pd_idx_to_midx, pd_dti_idx_date_only, filter_cols_below, reindex_on_time_mask, df_downsample_transpose, pd_single_ser, ser_shift, pd_common_idx_rows, df_midx_restack
from model.common import EXPECTED_NUM_HOURS

In [62]:
def prep_transpose_data(feature_df, row_masks_df, delayed=False):
	"""
	Converts a single indexed intraday DataFrame into a MultiIndexed daily DataFrame.

	Args:
		feature_df (pd.DataFrame): Intraday DataFrame
		row_masks_df (pd.DataFrame): DataFrame of row masks / time mask
		delayed (boolean): Whether or not to create a delayed function composition

	Returns:
		pd.DataFrame or dask Delayed object
	"""
	preproc = (
				reindex_on_time_mask,		# Converts the UTC time index to local time
				df_downsample_transpose,	# Performs the grouby downsample to daily frequency and intraday transpose
				filter_cols_below,		# Filters out columns with 90% or less of their data missing (relative to the most populated column)
				align_first_last_cols,		# Removes an extra column due to misalignment if it exists
				prune_nulls,			# Removes or fills any last null data
				pd_dti_idx_date_only,		# Removes the time component of the DatetimeIndex index
				df_midx_restack			# Restacks to fix https://github.com/pandas-dev/pandas/issues/2770
			)
	prep_fn = dcompose(*preproc) if (delayed) else compose(*preproc)
	return prep_fn(feature_df, row_masks_df)

In [63]:
res = prep_transpose_data(d, r)

In [64]:
res

9          10         11         12        13  \
id0        id1                                                                  
2009-01-02 pba_open       902.9900   900.8300   915.8100   916.9000   919.610   
           pba_high       906.6000   916.0100   919.0700   921.4900   925.730   
           pba_low        899.3500   900.1600   914.0000   916.3500   919.590   
           pba_close      901.3700   915.7000   916.8900   919.5200   924.100   
           pba_avgPrice   903.8950   908.2382   916.5228   918.7363   922.855   
...                            ...        ...        ...        ...       ...   
2017-12-29 pba_open      2689.1500  2687.8700  2685.1300  2685.7200  2684.320   
           pba_high      2692.1200  2687.8700  2686.5700  2686.3300  2685.030   
           pba_low       2686.8100  2683.3300  2683.2500  2683.9500  2681.370   
           pba_close     2687.8500  2685.1300  2685.7300  2684.3400  2682.230   
           pba_avgPrice  2688.7361  2686.3530  2684.6523  2685.3032  2683.679   

                                14         15         16  
id0        id1                                            
2009-01-02 pba_open       924.1300   928.3100   930.6000  
           pba_high       929.0500   934.7300   931.8000  
           pba_low        922.3200   928.3100   930.6000  
           pba_close      928.3400   930.4700   931.8000  
           pba_avgPrice   925.9128   931.0783   931.7599  
...                            ...        ...        ...  
2017-12-29 pba_open      2682.2400  2683.1500  2675.2200  
           pba_high      2683.3900  2685.8600  2675.9200  
           pba_low       2681.9500  2674.3600  2673.6100  
           pba_close     2683.1600  2675.1300  2673.6100  
           pba_avgPrice  2682.7271  2682.5202  2673.8399  

[11325 rows x 8 columns]

In [52]:
poi = []
if (asset_name == 'russell_2000'):
    poi.append(['2011-11-08', '2012-10-09', '2012-11-01', '2014-07-03', '2015-09-02', '2017-04-21'])
elif (asset_name == 'nasdaq_100'):
    poi.append(['2009-06-01', '2011-01-28', '2012-11-01'])
elif (asset_name == 'dow_jones'):
    poi.append(['2010-04-24'])

In [53]:
for point in poi:
    display(res.loc[point])

,,9,10,11,12,13,14,15,16
id0,id1,,,,,,,,


In [54]:
display(res.loc[res.iloc[:, 0].isnull()])

,,9,10,11,12,13,14,15,16
id0,id1,,,,,,,,


In [55]:
res.loc['2016-03-28']

9          10         11        12         13  \
id0        id1                                                                  
2016-03-28 pba_open      4415.0440  4399.0690  4402.0420  4408.443  4410.1090   
           pba_high      4417.5700  4413.2550  4409.4330  4412.063  4410.1410   
           pba_low       4393.0770  4396.4300  4397.7480  4401.787  4401.2770   
           pba_close     4399.0300  4402.0580  4408.5080  4410.123  4406.7220   
           pba_avgPrice  4400.8347  4404.4811  4404.1108  4406.987  4404.7084   

                                14         15        16  
id0        id1                                           
2016-03-28 pba_open      4406.7290  4396.9520  4398.348  
           pba_high      4413.6750  4406.8600  4398.348  
           pba_low       4394.2170  4394.7280  4398.066  
           pba_close     4396.8480  4398.3480  4398.066  
           pba_avgPrice  4405.1043  4401.8767  4398.066

In [56]:
res.loc['2009-01-02']

9          10         11        12        13  \
id0        id1                                                                 
2009-01-02 pba_open      1212.7400  1212.2800  1234.3200  1237.500  1244.190   
           pba_high      1216.3700  1235.1900  1238.9700  1246.660  1252.060   
           pba_low       1209.1100  1211.2600  1231.0400  1236.430  1243.470   
           pba_close     1212.3300  1234.6700  1237.5000  1244.090  1249.790   
           pba_avgPrice  1213.0478  1223.1257  1235.3515  1241.214  1247.801   

                                14         15      16  
id0        id1                                         
2009-01-02 pba_open      1249.8900  1255.5800  1263.7  
           pba_high      1257.7500  1266.4400  1263.7  
           pba_low       1247.2900  1255.5800  1263.7  
           pba_close     1255.4700  1261.1800  1263.7  
           pba_avgPrice  1252.7755  1259.6859  1263.7

In [57]:
d['2009-01-02']

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2009-01-02 14:00:00+00:00,1212.74,1216.37,1209.11,1212.33,1213.0478
2009-01-02 15:00:00+00:00,1212.28,1235.19,1211.26,1234.67,1223.1257
2009-01-02 16:00:00+00:00,1234.32,1238.97,1231.04,1237.50,1235.3515
2009-01-02 17:00:00+00:00,1237.50,1246.66,1236.43,1244.09,1241.2140
2009-01-02 18:00:00+00:00,1244.19,1252.06,1243.47,1249.79,1247.8010
2009-01-02 19:00:00+00:00,1249.89,1257.75,1247.29,1255.47,1252.7755
2009-01-02 20:00:00+00:00,1255.58,1266.44,1255.58,1261.18,1259.6859
2009-01-02 21:00:00+00:00,1263.70,1263.70,1263.70,1263.70,1263.7000
2009-01-02 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN


In [58]:
r['2009-01-02']

,times
id,
2009-01-02 00:00:00+00:00,NaT
2009-01-02 01:00:00+00:00,NaT
2009-01-02 02:00:00+00:00,NaT
2009-01-02 03:00:00+00:00,NaT
2009-01-02 04:00:00+00:00,NaT
2009-01-02 05:00:00+00:00,NaT
2009-01-02 06:00:00+00:00,NaT
2009-01-02 07:00:00+00:00,NaT
2009-01-02 08:00:00+00:00,NaT
